## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
import pandas as pd
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import shutil
import os
import pandas as pd 

In [2]:
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN','NAME'],axis=1,inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
for col in application_df.columns:
    print(f'{col:<30}',len(application_df[col].unique()))

APPLICATION_TYPE               17
AFFILIATION                    6
CLASSIFICATION                 71
USE_CASE                       5
ORGANIZATION                   4
STATUS                         2
INCOME_AMT                     9
SPECIAL_CONSIDERATIONS         2
ASK_AMT                        8747
IS_SUCCESSFUL                  2


In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
CUTOFF=520
application_types_to_replace=[]
for app in application_df['APPLICATION_TYPE'].unique():
    mask=application_df['APPLICATION_TYPE']==app
    cnt=application_df[mask]['APPLICATION_TYPE'].count()
    if cnt<CUTOFF:
        application_types_to_replace.append(app)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
col='CLASSIFICATION'
for v in application_df[col].unique():
    mask=application_df[col]==v
    cnt=application_df[mask][col].count()
    if cnt>1:
        print(f'{v:<30}',cnt)

C1000                          17326
C2000                          6074
C3000                          1918
C1200                          4837
C2700                          104
C7000                          777
C7200                          32
C1700                          287
C4000                          194
C7100                          75
C2800                          95
C6000                          15
C2100                          1883
C1238                          10
C5000                          116
C7120                          18
C1800                          15
C4100                          6
C1400                          34
C1270                          114
C2300                          32
C8200                          11
C1500                          16
C7210                          7
C1300                          58
C1230                          36
C1280                          50
C1240                          30
C2710                          3


In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
CUTOFF2=1883
classifications_to_replace=[]
col='CLASSIFICATION'
for v in application_df[col].unique():
    mask=application_df[col]==v
    cnt=application_df[mask][col].count()
    if cnt<CUTOFF2:
        classifications_to_replace.append(v)
#         print(f'{v:<30}',cnt)
        
# # Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df=pd.get_dummies(application_df)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [11]:
# Split our preprocessed data into our features and target arrays
y=application_df['IS_SUCCESSFUL']
application_df.drop('IS_SUCCESSFUL',axis=1,inplace=True)
X=application_df
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1,stratify=y)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn=tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=43))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu", input_dim=45))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5981 (23.36 KB)
Trainable params: 5981 (23.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
804/804 [==============================] - 12s 9ms/step - loss: 0.5741 - accuracy: 0.7194
Epoch 2/5
804/804 [==============================] - 8s 10ms/step - loss: 0.5564 - accuracy: 0.7303
Epoch 3/5
804/804 [==============================] - 7s 9ms/step - loss: 0.5529 - accuracy: 0.7306
Epoch 4/5
804/804 [==============================] - 7s 9ms/step - loss: 0.5517 - accuracy: 0.7298
Epoch 5/5
804/804 [==============================] - 6s 7ms/step - loss: 0.5499 - accuracy: 0.7311


In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5589 - accuracy: 0.7235 - 1s/epoch - 4ms/step
Loss: 0.5588974952697754, Accuracy: 0.723498523235321


In [17]:
# Export our model to HDF5 file
nn.save_weights('model01.hdf5')

# Optimize Model

### Shrink the values some more

In [19]:
dirpath=('./untitled_project')
shutil.rmtree(dirpath)
os.mkdir(dirpath)

FileNotFoundError: [WinError 3] The system cannot find the path specified: './untitled_project'

#### parameters

In [ ]:
COLUMN_ADJUSTMENTS=[['APPLICATION_TYPE',800,'Other'],
                    ['CLASSIFICATION',2000,'Other'],
                   ]
COLUMNS_TO_KEEP=10
#tuner parameters
EPOCHS=10
ITERATIONS=5
MAX_LAYERS=10
MAX_NODES=20

In [4]:
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
COLUMNS_TO_DROP=['EIN','NAME']
try:
    application_df.drop(COLUMNS_TO_DROP,axis=1,inplace=True)
except: pass
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
def bin_small_items(col='CLASSIFICATION',cutoff=1883,desc='Other'):
    classifications_to_replace=[]
#     col='CLASSIFICATION'
    for v in application_df[col].unique():
        mask=application_df[col]==v
        cnt=application_df[mask][col].count()
        if cnt<cutoff:
            classifications_to_replace.append(v)
    #         print(f'{v:<30}',cnt)

    # # Replace in dataframe
    for cls in classifications_to_replace:
        application_df[col] = application_df[col].replace(cls,desc)    
    # Check to make sure binning was successful
    print(application_df[col].value_counts())
for spec in COLUMN_ADJUSTMENTS:
    col,cutoff,desc=spec
    bin_small_items(col,cutoff,desc)

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64
C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64


#### drop highly correlated columns

In [6]:
# application_df.corr()
cor_matrix = application_df.corr().abs() 
# print(cor_matrix)
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool)) 
# print(upper_tri)
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)] 
# print(to_drop)
# application_df.drop(application_df.columns[to_drop],axis=1,inplace=True)
application_df.drop(to_drop,axis=1,inplace=True)
application_df.head()

C:\Users\rober\AppData\Local\Temp\ipykernel_9040\740307613.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  cor_matrix = application_df.corr().abs()
C:\Users\rober\AppData\Local\Temp\ipykernel_9040\740307613.py:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,Other,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,Other,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


#### encode categorical columns and then keep only the top 15 most important

In [7]:
def drop_more_columns(X,y):
    k=COLUMNS_TO_KEEP
    X_new = SelectKBest(f_regression, k=k).fit_transform(X, y)
    selected_features = X.columns[SelectKBest(f_regression, k=k).fit(X, y).get_support()]
    delete_these = set(application_df.columns)-set(selected_features)
    delete_these=list(delete_these)
    for col in delete_these:
        print('deleting %s'%col)
        X.drop(col,axis=1,inplace=True)

In [8]:
for col in application_df.columns:
    print(f'{col:<30}',len(application_df[col].unique()))

APPLICATION_TYPE               6
AFFILIATION                    6
CLASSIFICATION                 4
USE_CASE                       5
ORGANIZATION                   4
STATUS                         2
INCOME_AMT                     9
SPECIAL_CONSIDERATIONS         2
ASK_AMT                        8747
IS_SUCCESSFUL                  2


In [9]:
application_df=pd.get_dummies(application_df)
# Split our preprocessed data into our features and target arrays
y=application_df['IS_SUCCESSFUL']
application_df.drop('IS_SUCCESSFUL',axis=1,inplace=True)
X=application_df
#try removing move
drop_more_columns(X,y)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1,stratify=y)
# Create a StandardScaler instances
scaler = StandardScaler()
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

deleting INCOME_AMT_5M-10M
deleting AFFILIATION_Other
deleting APPLICATION_TYPE_T5
deleting APPLICATION_TYPE_T19
deleting ORGANIZATION_Co-operative
deleting SPECIAL_CONSIDERATIONS_Y
deleting APPLICATION_TYPE_Other
deleting INCOME_AMT_50M+
deleting ASK_AMT
deleting USE_CASE_CommunityServ
deleting CLASSIFICATION_C2000
deleting SPECIAL_CONSIDERATIONS_N
deleting AFFILIATION_Family/Parent
deleting AFFILIATION_Regional
deleting ORGANIZATION_Corporation
deleting CLASSIFICATION_C1200
deleting ORGANIZATION_Trust
deleting USE_CASE_Preservation
deleting INCOME_AMT_1-9999
deleting STATUS
deleting INCOME_AMT_25000-99999
deleting APPLICATION_TYPE_T3
deleting APPLICATION_TYPE_T4
deleting USE_CASE_ProductDev
deleting INCOME_AMT_1M-5M
deleting CLASSIFICATION_Other
deleting ORGANIZATION_Association
deleting USE_CASE_Other
deleting CLASSIFICATION_C1000
deleting INCOME_AMT_100000-499999
deleting AFFILIATION_National
deleting INCOME_AMT_10M-50M
deleting APPLICATION_TYPE_T6
deleting USE_CASE_Heathcare
delet

In [10]:
application_df.shape

(34299, 1)

### Run the optimizer

In [11]:
# Create a method that creates a new Sequential model with hyperparameter options
INPUT_DIM=X_train_scaled.shape[1]
def create_model(hp):
    nn_model = tf.keras.models.Sequential()
    activation = hp.Choice('activation',['relu','tanh','sigmoid','softmax'])
    hp_optimizer = hp.Choice('optimizer', values=['sgd', 'rmsprop', 'adam'])
    hp_lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    if hp_optimizer == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=hp_lr)
    elif hp_optimizer == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=hp_lr)
    elif hp_optimizer == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=hp_lr)
    else:
        raise
    
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=MAX_NODES,
        step=2), activation=activation, input_dim=INPUT_DIM))
    for i in range(hp.Int('num_layers', 1, MAX_LAYERS)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=MAX_NODES,
            step=2),
            activation=activation))
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    nn_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return nn_model

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=EPOCHS,
    hyperband_iterations=ITERATIONS)
print(X_train_scaled.shape,y_train.shape,X_test_scaled.shape,y_test.shape)

(25724, 1) (25724,) (8575, 1) (8575,)


In [12]:
tuner.search(X_train_scaled,y_train,epochs=EPOCHS,validation_data=(X_test_scaled,y_test))

Trial 41 Complete [00h 00m 08s]
val_accuracy: 0.5323615074157715

Best val_accuracy So Far: 0.6880466341972351
Total elapsed time: 00h 07m 33s

Search: Running Trial #42

Value             |Best Value So Far |Hyperparameter
softmax           |relu              |activation
sgd               |rmsprop           |optimizer
0.01              |0.001             |learning_rate
17                |13                |first_units
10                |5                 |num_layers
9                 |9                 |units_0
9                 |9                 |units_1
19                |9                 |units_2
15                |13                |units_3
7                 |17                |units_4
11                |9                 |units_5
5                 |11                |units_6
7                 |None              |units_7
3                 |None              |units_8
13                |None              |units_9
2                 |2                 |tuner/epochs
0                

In [13]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [14]:
X_test_scaled[:5]

In [15]:
y_test[:5]

In [16]:
best_hyper.values